# **Task 1: Implement the Diffie-Hellman Key Exchange**
Write a Python function to:
- Generate a large prime number p and a primitive root g.
- Accept private keys a and b, compute public values A and B.
- Exchange public keys and compute the shared secret.
- Verify that both parties derive the same secret key.

In [ ]:
import random

# Step 1: Generate a large prime number `p` and its primitive root `g`.
def is_prime(n):
    """Check if a number is prime."""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5)+1, 2):
        if n % i == 0:
            return False
    return True

def find_primitive_root(p):
    """Find a primitive root modulo p."""
    if not is_prime(p):
        return None
    phi = p - 1
    factors = set()
    n = phi
    i = 2
    while i * i <= n:
        if n % i == 0:
            factors.add(i)
            while n % i == 0:
                n //= i
        i += 1
    if n > 1:
        factors.add(n)

    for g in range(2, p):
        flag = False
        for factor in factors:
            if pow(g, phi // factor, p) == 1:
                flag = True
                break
        if not flag:
            return g
    return None

def generate_large_prime():
    """Generate a large random prime number."""
    while True:
        p = random.randint(1000, 5000)
        if is_prime(p):
            return p

# Step 2: Define the key exchange logic
def diffie_hellman_key_exchange(a, b):
    # Generate prime p and primitive root g
    p = generate_large_prime()
    g = find_primitive_root(p)

    print(f"Prime number (p): {p}")
    print(f"Primitive root (g): {g}")

    # Compute public values
    A = pow(g, a, p)  # A = g^a mod p
    B = pow(g, b, p)  # B = g^b mod p

    print(f"\nPrivate key of Alice (a): {a}")
    print(f"Private key of Bob (b): {b}")
    print(f"Alice sends public value (A): {A}")
    print(f"Bob sends public value (B): {B}")

    # Shared secret computed independently
    secret_alice = pow(B, a, p)  # (B^a mod p)
    secret_bob = pow(A, b, p)    # (A^b mod p)

    print(f"\nShared secret computed by Alice: {secret_alice}")
    print(f"Shared secret computed by Bob: {secret_bob}")

    # Verify both secrets match
    if secret_alice == secret_bob:
        print("\n✅ Key exchange successful. Both parties share the same secret.")
    else:
        print("\n❌ Key exchange failed. Secrets do not match.")

# Example usage:
a = random.randint(2, 100)
b = random.randint(2, 100)
diffie_hellman_key_exchange(a, b)


Prime number (p): 2897
Primitive root (g): 3

Private key of Alice (a): 29
Private key of Bob (b): 49
Alice sends public value (A): 1452
Bob sends public value (B): 1090

Shared secret computed by Alice: 1239
Shared secret computed by Bob: 1239

✅ Key exchange successful. Both parties share the same secret.


# **Task 2: Key Agreement with Different Prime Sizes**
- Repeat Task 1 using different sizes of prime numbers (e.g., 8-bit, 16-bit, 32-bit, 64-bit).
- Measure time taken for key computation.
- Analyze how prime size affects computation time and security.

In [ ]:
import random
import time
from sympy import isprime, nextprime

# Utility: Generate a random prime of approx `bit_size` bits
def generate_prime(bit_size):
    start = 2**(bit_size - 1)
    end = 2**bit_size - 1
    candidate = random.randint(start, end)
    prime = nextprime(candidate)
    return prime

# Utility: Find a primitive root modulo p (simple version for small primes)
def find_primitive_root(p):
    phi = p - 1
    for g in range(2, p):
        powers = set(pow(g, power, p) for power in range(1, phi))
        if len(powers) == phi - 1:
            return g
    return None

# Diffie-Hellman with timing
def diffie_hellman_timed(bit_size, a, b):
    print(f"\n🔐 Running Diffie-Hellman with {bit_size}-bit prime:")

    # Generate prime and primitive root
    start_time = time.time()
    p = generate_prime(bit_size)
    g = find_primitive_root(p)
    if g is None:
        print("⚠️ Could not find a primitive root. Try again.")
        return
    keygen_time = time.time() - start_time

    # Public keys
    start_time = time.time()
    A = pow(g, a, p)
    B = pow(g, b, p)
    public_time = time.time() - start_time

    # Shared secrets
    start_time = time.time()
    secret1 = pow(B, a, p)
    secret2 = pow(A, b, p)
    shared_time = time.time() - start_time

    # Results
    success = secret1 == secret2
    print(f"Prime (p): {p}")
    print(f"Primitive root (g): {g}")
    print(f"Shared secret match: {'✅ Yes' if success else '❌ No'}")
    print(f"Time to generate key params: {keygen_time:.6f}s")
    print(f"Time to compute public keys: {public_time:.6f}s")
    print(f"Time to compute shared secret: {shared_time:.6f}s")
    return keygen_time, public_time, shared_time

# Run for multiple bit sizes
def test_dh_across_sizes():
    bit_sizes = [8, 16, 32, 64]
    results = []

    # Random private keys (static for comparison)
    a = random.randint(100, 1000)
    b = random.randint(100, 1000)

    for size in bit_sizes:
        result = diffie_hellman_timed(size, a, b)
        if result:
            results.append((size, *result))

    print("\n📊 Summary:")
    print("Bits | KeyGen Time | Public Key Time | Shared Secret Time")
    for row in results:
        print(f"{row[0]:>4} | {row[1]:>12.6f}s | {row[2]:>15.6f}s | {row[3]:>18.6f}s")

test_dh_across_sizes()



🔐 Running Diffie-Hellman with 8-bit prime:
Prime (p): 179
Primitive root (g): 2
Shared secret match: ✅ Yes
Time to generate key params: 0.000114s
Time to compute public keys: 0.000001s
Time to compute shared secret: 0.000001s

🔐 Running Diffie-Hellman with 16-bit prime:
Prime (p): 57751
Primitive root (g): 6
Shared secret match: ✅ Yes
Time to generate key params: 0.315945s
Time to compute public keys: 0.000007s
Time to compute shared secret: 0.000002s

🔐 Running Diffie-Hellman with 32-bit prime:


# **Task 3: Man-in-the-Middle Simulation**
- Simulate a scenario where a malicious actor intercepts the public keys and substitutes them.
- Demonstrate how the attacker can derive a different shared secret with each party.
- Discuss how authenticated Diffie-Hellman can prevent this attack.

Simulation Plan
Alice and Bob want to communicate securely using Diffie-Hellman.

Mallory (the attacker) intercepts their public keys.

Mallory substitutes his own public key with each party.

Alice and Bob think they're talking directly—but they're actually talking to Mallory.

Mallory ends up with two shared secrets:

One with Alice.

One with Bob.

In [ ]:
import random
from sympy import nextprime

# Prime generation (small for clarity)
def generate_prime(bit_size=16):
    start = 2**(bit_size - 1)
    return nextprime(random.randint(start, 2**bit_size - 1))

def find_primitive_root(p):
    phi = p - 1
    for g in range(2, p):
        powers = set(pow(g, power, p) for power in range(1, phi))
        if len(powers) == phi - 1:
            return g
    return None

# Simulate MitM Attack
def mitm_attack():
    # Generate shared prime and generator
    p = generate_prime(16)
    g = find_primitive_root(p)

    print(f"🧮 Shared prime p: {p}")
    print(f"🧮 Generator g: {g}\n")

    # Private keys
    a = random.randint(100, 200)  # Alice
    b = random.randint(100, 200)  # Bob
    m1 = random.randint(100, 200) # Mallory <-> Alice
    m2 = random.randint(100, 200) # Mallory <-> Bob

    # Alice computes public key
    A = pow(g, a, p)
    # Bob computes public key
    B = pow(g, b, p)

    # Mallory intercepts and substitutes:
    M1 = pow(g, m1, p)  # To Bob instead of A
    M2 = pow(g, m2, p)  # To Alice instead of B

    print(f"Alice sends public A: {A} -> Mallory intercepts -> Bob receives: {M1}")
    print(f"Bob sends public B: {B} -> Mallory intercepts -> Alice receives: {M2}\n")

    # Each party computes what they think is the shared key
    alice_secret = pow(M2, a, p)       # Alice with Mallory's M2
    bob_secret = pow(M1, b, p)         # Bob with Mallory's M1

    # Mallory computes both secrets
    mallory_secret_alice = pow(A, m2, p)
    mallory_secret_bob = pow(B, m1, p)

    print(f"🔐 Alice computes shared secret with fake B: {alice_secret}")
    print(f"🔐 Bob computes shared secret with fake A: {bob_secret}")
    print(f"🕵️ Mallory computes secret with Alice: {mallory_secret_alice}")
    print(f"🕵️ Mallory computes secret with Bob:   {mallory_secret_bob}")

    if alice_secret != bob_secret:
        print("\n⚠️ Alice and Bob do NOT share the same secret. Communication compromised.")
    else:
        print("\n✅ Shared secret match (no attack).")

mitm_attack()


🧮 Shared prime p: 39821
🧮 Generator g: 2

Alice sends public A: 29878 -> Mallory intercepts -> Bob receives: 18880
Bob sends public B: 37274 -> Mallory intercepts -> Alice receives: 35619

🔐 Alice computes shared secret with fake B: 27144
🔐 Bob computes shared secret with fake A: 6382
🕵️ Mallory computes secret with Alice: 27144
🕵️ Mallory computes secret with Bob:   6382

⚠️ Alice and Bob do NOT share the same secret. Communication compromised.


# **Task 4: Security Analysis and Report**
- Discuss the security of Diffie-Hellman against brute-force and logarithmic attacks (finding
discrete logarithm).
-- Explain the importance of choosing large prime numbers and secure parameters.

The Diffie-Hellman key exchange is secure because it is very hard to reverse the process of calculating the shared key when the right parameters are used. An attacker cannot easily guess the private key if the values involved are large and chosen properly.

Brute-force attacks, which try all possible keys, are not practical if the key is long enough. More advanced attacks, like Pollard’s Rho or the Number Field Sieve, can break the system more efficiently—but only if the prime number used is small.

This is why using a large prime number is important. A 512-bit prime is no longer secure and can be broken quickly. A 1024-bit prime is also becoming weak. For safe communication today, at least a 2048-bit prime should be used.

If the prime or generator is weak or reused, or if private keys are not random enough, attackers have a better chance of breaking the system. Also, if the public keys are not verified, a man-in-the-middle attacker can intercept and change them.

To stay secure, it is important to use strong, large values, generate private keys properly, and always verify public keys before using them.